In [1]:
## Self attention head math
import torch
import torch.nn as nn
from torch.nn import functional as F

In [2]:
## Out inputs are B, T, C i.e. Batch, Time, Channel (Vocab size)
B = 4
T = 8 # Number of timesteps i.e. longest name makemore names -> 15
C = 32 # Number of alphabets i.e. makemore -> 26

x = torch.randn(B,T,C)

In [3]:
## We need to initialize 3 parts to self attention head: Keys, Queries, Values
# Keys are basically broadcast saying - I have this information i.e. I am consonant
# Queries are broadcast saying - Who has information i.e. I need a vowel
# Value is what information is passed from the key to the query
# Seems like an extermely genric mechanism for information sharing
# Unclear what defines how many heads should be used or what the head size should be. In Karpathy example its 16.

head_size = 16
key = nn.Linear(C, head_size, bias=False)
query = nn.Linear(C, head_size, bias=False)
value = nn.Linear(C, head_size, bias=False)

In [4]:
## Initialize the vector
k = key(x) # B,T,head_size
q = query(x) # B,T,head_size
v = value(x) # B,T,head_size

In [5]:
## Get scaled self attention weights

# Calculate wei
wei = k @ torch.transpose(q, 1, 2) * head_size**-0.5 # B,T,T

# Mask all the future tokens from commuicating i.e. make it causal 
# 1. Create lower triangular matrix and use it to mask wei
ltmat = torch.tril(torch.ones(T,T))
wei = torch.masked_fill(wei, ltmat == 0, float('-Inf'))
# 2. Now do a softmax per row to make normalize to 1
wei = F.softmax(wei,dim=-1)

# Check that each row of every batch is normalized to 1
# print(sum(wei[0, 0,:]))

In [6]:
# Finally calculate output value of self attention block

output = wei @ v # B, T, head_size
output.shape

torch.Size([4, 8, 16])